In [1]:
!nvidia-smi

Sat Dec 18 15:38:09 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P0    24W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [15]:
import xgboost as xgb
import sklearn

from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import log_loss,confusion_matrix,classification_report,roc_curve,auc,accuracy_score,roc_auc_score
from sklearn.model_selection import StratifiedKFold
# Onehot encoding

from sklearn.preprocessing import LabelEncoder

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from scipy import sparse
import gc
import os
import random

In [3]:
def seed_everything(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    #torch.manual_seed(seed)
    #torch.cuda.manual_seed(seed)
    #torch.backends.cudnn.deterministic = True
    #torch.backends.cudnn.benchmark = False
seed_everything()

In [4]:

!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00359/NewsAggregatorDataset.zip
!unzip NewsAggregatorDataset.zip


--2021-12-18 15:38:18--  https://archive.ics.uci.edu/ml/machine-learning-databases/00359/NewsAggregatorDataset.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 29224203 (28M) [application/x-httpd-php]
Saving to: ‘NewsAggregatorDataset.zip’

NewsAggregatorDatas 100%[===================>]  27.87M  11.1MB/s    in 2.5s    

2021-12-18 15:38:21 (11.1 MB/s) - ‘NewsAggregatorDataset.zip’ saved [29224203/29224203]

Archive:  NewsAggregatorDataset.zip
  inflating: 2pageSessions.csv       
   creating: __MACOSX/
  inflating: __MACOSX/._2pageSessions.csv  
  inflating: newsCorpora.csv         
  inflating: __MACOSX/._newsCorpora.csv  
  inflating: readme.txt              
  inflating: __MACOSX/._readme.txt   


In [5]:
# 読込時のエラー回避のためダブルクォーテーションをシングルクォーテーションに置換
!sed -e 's/"/'\''/g' ./newsCorpora.csv > ./newsCorpora_re.csv

In [6]:
# データの読込
df = pd.read_csv('./newsCorpora_re.csv', header=None, sep='\t', names=['ID', 'TITLE', 'URL', 'PUBLISHER', 'CATEGORY', 'STORY', 'HOSTNAME', 'TIMESTAMP'])

# データの抽出
df = df.loc[df['PUBLISHER'].isin(['Reuters', 'Huffington Post', 'Businessweek', 'Contactmusic.com', 'Daily Mail']), ['TITLE', 'CATEGORY']]


In [7]:
df.head()


,TITLE,CATEGORY
12,Europe reaches crunch point on banking union,b
13,ECB FOCUS-Stronger euro drowns out ECB's messa...,b
19,"Euro Anxieties Wane as Bunds Top Treasuries, S...",b
20,Noyer Says Strong Euro Creates Unwarranted Eco...,b
29,REFILE-Bad loan triggers key feature in ECB ba...,b


In [8]:
# データの分割
df_train, df_valid_test = train_test_split(df, test_size=0.2, shuffle=True, random_state=123, stratify=df['CATEGORY'])
df_valid, df_test = train_test_split(df_valid_test, test_size=0.5, shuffle=True, random_state=123, stratify=df_valid_test['CATEGORY'])
df_train.reset_index(drop=True, inplace=True)
df_valid.reset_index(drop=True, inplace=True)
df_test.reset_index(drop=True, inplace=True)

print(df_train.head())

                                               TITLE CATEGORY
0  REFILE-UPDATE 1-European car sales up for sixt...        b
1  Amazon Plans to Fight FTC Over Mobile-App Purc...        t
2  Kids Still Get Codeine In Emergency Rooms Desp...        m
3  What On Earth Happened Between Solange And Jay...        e
4  NATO Missile Defense Is Flight Tested Over Hawaii        b


In [9]:
le = LabelEncoder()
y_tr = le.fit_transform(df_train['CATEGORY'].values)
y_vl = le.transform(df_valid['CATEGORY'].values)
y_te = le.transform(df_test['CATEGORY'].values)

In [10]:
!pip install transformers

     |████████████████████████████████| 3.4 MB 14.8 MB/s 
     |████████████████████████████████| 61 kB 505 kB/s 
     |████████████████████████████████| 895 kB 77.1 MB/s 
     |████████████████████████████████| 3.3 MB 72.5 MB/s 
     |████████████████████████████████| 596 kB 67.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [11]:
import torch
import transformers

from transformers import BertTokenizer


class BertSequenceVectorizer:
    def __init__(self, model_name="bert-base-uncased", max_len=128):
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.model_name = model_name
        self.tokenizer = BertTokenizer.from_pretrained(self.model_name)
        self.bert_model = transformers.BertModel.from_pretrained(self.model_name)
        self.bert_model = self.bert_model.to(self.device)
        self.max_len = max_len

    def vectorize(self, sentence: str) -> np.array:
        inp = self.tokenizer.encode(sentence)
        len_inp = len(inp)

        if len_inp >= self.max_len:
            inputs = inp[:self.max_len]
            masks = [1] * self.max_len
        else:
            inputs = inp + [0] * (self.max_len - len_inp)
            masks = [1] * len_inp + [0] * (self.max_len - len_inp)

        inputs_tensor = torch.tensor([inputs], dtype=torch.long).to(self.device)
        masks_tensor = torch.tensor([masks], dtype=torch.long).to(self.device)

        bert_out = self.bert_model(inputs_tensor, masks_tensor)
        seq_out, pooled_out = bert_out['last_hidden_state'], bert_out['pooler_output']

        if torch.cuda.is_available():    
            return seq_out[0][0].cpu().detach().numpy() # 0番目は [CLS] token, 768 dim の文章特徴量
        else:
            return seq_out[0][0].detach().numpy()

In [12]:
BSV = BertSequenceVectorizer(
    model_name="bert-base-uncased",
    max_len=128)

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [13]:
# Word vector
tr_vect = np.stack(df_train["TITLE"].fillna("").map(lambda x: BSV.vectorize(x).reshape(-1)).values)
vl_vect = np.stack(df_valid["TITLE"].fillna("").map(lambda x: BSV.vectorize(x).reshape(-1)).values)
ts_vect = np.stack(df_test["TITLE"].fillna("").map(lambda x: BSV.vectorize(x).reshape(-1)).values)

In [14]:
np.save('tr_vect.npy', tr_vect)
np.save('vl_vect.npy', vl_vect)
np.save('ts_vect.npy', ts_vect)

In [16]:
model = xgb.XGBClassifier()
model.fit(tr_vect, y_tr)

XGBClassifier(objective='multi:softprob')

In [17]:
# 検証データを予測する
y_pred = model.predict_proba(vl_vect)
y_pred_max = np.argmax(y_pred, axis=1)  # 最尤と判断したクラスの値にする

accuracy = sum(y_vl == y_pred_max) / len(y_vl)
print('{:.4f}'.format(accuracy))
print('{:.4f}'.format(roc_auc_score(y_vl, y_pred, multi_class='ovo')))
print('{:.4f}'.format(log_loss(y_vl, y_pred)))

# 評価データを予測する
y_pred = model.predict_proba(ts_vect)
y_pred_max = np.argmax(y_pred, axis=1)  # 最尤と判断したクラスの値にする

accuracy = sum(y_te == y_pred_max) / len(y_te)
print('{:.4f}'.format(accuracy))
print('{:.4f}'.format(roc_auc_score(y_te, y_pred, multi_class='ovo')))
print('{:.4f}'.format(log_loss(y_te, y_pred)))

0.8593
0.9414
0.3942
0.8623
0.9472
0.3852
